# Close the Budgets

In [174]:
import fastjmd95 # documentation here: https://github.com/xgcm/fastjmd95
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from importlib import reload
import os
from xmitgcm import open_mdsdataset


from analysis_package import plotting_functions
from analysis_package import open_datasets
from analysis_package import ecco_masks
from analysis_package import integrate_zonally
from analysis_package import lat_fields
from analysis_package import convert_to_sigspace

import sys
sys.path.append('/Users/tatsumonkman/ECCOv4-py/ECCOv4-py')
import ecco_v4_py as ecco


from analysis_package import plotting_functions
from analysis_package import open_datasets

open_datasets = reload(open_datasets)
convert_to_sigspace = reload(convert_to_sigspace)

In [175]:
######################################################################################################################
################################################### LOAD GRID ########################################################
######################################################################################################################
grid_path = "./ecco_grid/ECCOv4r3_grid.nc"
grid = xr.open_dataset(grid_path)
cds = grid.coords.to_dataset()
grid_xmitgcm = ecco.ecco_utils.get_llc_grid(cds)

######################################################################################################################
############################################# CREATE DOMAIN MASKS ####################################################
######################################################################################################################
maskW = xr.open_dataarray("generic_masks/maskW.nc")
maskS = xr.open_dataarray("generic_masks/maskS.nc")
maskC = xr.open_dataarray("generic_masks/maskC.nc")

southern_ocean_mask_W, southern_ocean_mask_S, southern_ocean_mask_C, so_atl_basin_mask_W, so_atl_basin_mask_S, so_atl_basin_mask_C, so_indpac_basin_mask_W, so_indpac_basin_mask_S, so_indpac_basin_mask_C = ecco_masks.get_basin_masks(maskW, maskS, maskC)

baffin_mask_C = ecco.get_basin_mask("baffin",maskC)
north_mask_C = ecco.get_basin_mask("north",maskC)
hudson_mask_C = ecco.get_basin_mask("hudson",maskC)
gin_mask_C = ecco.get_basin_mask("gin",maskC)
bering_mask_C = ecco.get_basin_mask("bering",maskC)
okhotsk_mask_C = ecco.get_basin_mask("okhotsk",maskC)
atl_mask_C = ecco.get_basin_mask("atl",maskC)

maskC_low_depth_filtered = maskC*maskC.where(grid.Depth > 50.)

# masks with low depths filtered out
so_atl_maskC_low_depth_filtered = so_atl_basin_mask_C*maskC.where(grid.Depth > 100.)
so_indpac_maskC_low_depth_filtered = so_indpac_basin_mask_C*maskC.where(grid.Depth > 100.)





mexico_maskC = ecco.get_basin_mask("mexico",maskC)
mexico_maskC = mexico_maskC.where(mexico_maskC!=0,other=np.nan)
#plotting_functions.world_plot(mexico_maskC)
#plotting_functions.world_plot(so_atl_basin_mask_C.where(mexico_maskC!=1))
# Atlantic mask no caribbean
so_atl_nomex_basin_mask_C = so_atl_basin_mask_C.where(mexico_maskC!=1)

atl_midlat_basin_mask_C = so_atl_basin_mask_C.where(so_atl_basin_mask_C.lat<=50,other=np.nan).where(so_atl_basin_mask_C.lat>=-32,other=np.nan)
indpac_basin_mask_C = so_indpac_basin_mask_C.where(so_indpac_basin_mask_C.lat>=-32,other=np.nan)
so_basin_mask_C = so_indpac_basin_mask_C.where(so_indpac_basin_mask_C.lat<-32,other=np.nan)


so_atl_maskC_low_depth_filtered_noGF = so_atl_maskC_low_depth_filtered.copy()
so_atl_maskC_low_depth_filtered_noGF.loc[{"tile":10}][30:74,40:60] = np.nan
so_atl_maskC_low_depth_filtered_noGF.loc[{"tile":10}][65:76,45:55] = np.nan
so_atl_maskC_low_depth_filtered_noGF.loc[{"tile":10}][30:60,50:65] = np.nan


atl_arct_basin_mask_C = so_atl_maskC_low_depth_filtered.fillna(0) + ecco.get_basin_mask("arct",maskC) + ecco.get_basin_mask("barents",maskC)
atl_arct_basin_mask_C = atl_arct_basin_mask_C.where(atl_arct_basin_mask_C>0,other=np.nan)
atl_nogin_basin_mask_C = so_atl_maskC_low_depth_filtered.where(so_atl_basin_mask_C.lat>-32,other=np.nan).where((gin_mask_C+north_mask_C < 1).values,other=np.nan)
atl_nogin_basin_mask_C_special_mask = atl_nogin_basin_mask_C.copy()


FINAL_atl_maskC = so_atl_maskC_low_depth_filtered_noGF*atl_nogin_basin_mask_C_special_mask




load_binary_array: loading file /Users/tatsumonkman/3rd_party_software/ECCOv4-py/binary_data/basins.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
shape after reading 
(13, 90, 90)
load_binary_array: loading file /Users/tatsumonkman/3rd_party_software/ECCOv4-py/binary_data/basins.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
shape after reading 
(13, 90, 90)
load_binary_array: loading file /Users/tatsumonkman/3rd_party_software/ECCOv4-py/binary_data/basins.data
load_binary_array: data array shape  (1170, 90)
load_binary

In [176]:

atl_nogin_basin_mask_C = so_atl_basin_mask_C.where(so_atl_basin_mask_C.lat>-32,other=np.nan).where((gin_mask_C+north_mask_C < 1).values,other=np.nan)

so_atl_nogin_basin_mask_C = so_atl_basin_mask_C.where((gin_mask_C+north_mask_C < 1).values,other=np.nan)




atl_nogin_basin_mask_C = so_atl_basin_mask_C.where((gin_mask_C+north_mask_C < 1).values,other=np.nan)
atl_nogin_basin_mask_C_special_mask = atl_nogin_basin_mask_C.copy()

FINAL_atl_maskC = so_atl_maskC_low_depth_filtered_noGF*atl_nogin_basin_mask_C_special_mask



i_range0 = np.arange(0,15)
j_range0 = np.arange(76,90)

i_range1 = np.arange(20,35)
j_range1 = np.arange(60,80)

i_range2 = np.arange(10,20)
j_range2 = np.arange(73,78)


mod_FINAL_atl_maskC = FINAL_atl_maskC.copy()

region_0_mask = (SIGMA2_tmax.isel(tile=2,i=i_range0,j=j_range0).max(dim="k")>1036.98).fillna(0)
region_1_mask = (SIGMA2_tmax.isel(tile=2,i=i_range1,j=j_range1).max(dim="k")>1036.98).fillna(0)
region_2_mask = (SIGMA2_tmax.isel(tile=2,i=i_range2,j=j_range2).max(dim="k")>1036.98).fillna(0)
region_0_mask = region_0_mask.where(region_0_mask<1,other=np.nan)*0+1
region_1_mask = region_1_mask.where(region_1_mask<1,other=np.nan)*0+1
region_2_mask = region_2_mask.where(region_2_mask<1,other=np.nan)*0+1


mod_FINAL_atl_maskC.loc[{"tile":2,"i":i_range0,"j":j_range0}] = mod_FINAL_atl_maskC.loc[{"tile":2,"i":i_range0,"j":j_range0}].values*region_0_mask.values
mod_FINAL_atl_maskC.loc[{"tile":2,"i":i_range1,"j":j_range1}] = mod_FINAL_atl_maskC.loc[{"tile":2,"i":i_range1,"j":j_range1}].values*region_1_mask.values
mod_FINAL_atl_maskC.loc[{"tile":2,"i":i_range2,"j":j_range2}] = mod_FINAL_atl_maskC.loc[{"tile":2,"i":i_range2,"j":j_range2}].values*region_2_mask.values



In [180]:
ds_llc = open_mdsdataset('global_oce_llc90/', iters=8, geometry='llc')

In [181]:
SIGMA2_k0_tmax_test = (SIGMA2_k0_tmax.where(SIGMA2_k0_tmax.lat >= 30, other=np.nan)*mod_FINAL_atl_maskC).copy()



In [275]:
class nf(float):
    def __repr__(self):
        s = f'{self:.1f}'
        return f'{self:.0f}' if s[-1] == '0' else s



In [183]:
SIGMA2 = xr.open_dataarray(f"./SIGMA2_full.nc")
SIGMA2_ktmax = (SIGMA2.max(dim=["time","k"]).assign_coords(maskC.coords)*maskC)


SIGMA2_k0_tmax = (SIGMA2.isel(k=0).max(dim="time").assign_coords(maskC.coords)*maskC)
SIGMA2_k0_tmax_SH = SIGMA2_k0_tmax.where(SIGMA2_k0_tmax.lat <= -20, other=np.nan)
SIGMA2_k0_tmax_NH = SIGMA2_k0_tmax.where(SIGMA2_k0_tmax.lat >= 30, other=np.nan)*mod_FINAL_atl_maskC
SIGMA2_ktmax_NH = SIGMA2_ktmax.where(SIGMA2_ktmax.lat >= 0, other=np.nan)*mod_FINAL_atl_maskC
SIGMA2_k0_tmax_NH_ip = SIGMA2_k0_tmax.where(SIGMA2_k0_tmax.lat >= 30, other=np.nan)*so_indpac_basin_mask_C



SIGMA2.close()

# make a blank mask of 1's
blank_mask = xr.open_dataarray("Good_Outputs/T_TH_advection_1.nc").assign_coords(maskC.coords)*0+1
SO_sig_mask = blank_mask.copy()


for lat in np.arange(-88,-20):
    
    SIGMA2_k0_tmax_SH_tmp = SIGMA2_k0_tmax_SH.where(SIGMA2_k0_tmax_SH.lat >= lat,other=np.nan).copy()
    sig_max = SIGMA2_k0_tmax_SH_tmp.max()
    print(lat, sig_max.values)

    tmp_mask = blank_mask.copy()*0
    tmp_mask = tmp_mask.where(tmp_mask.sig < sig_max, other=np.nan)
    tmp_mask = tmp_mask.where(tmp_mask.lat < lat, other=np.nan)
    tmp_mask = tmp_mask.fillna(1)
    
    SO_sig_mask = SO_sig_mask*tmp_mask



-88 1037.8419949637437
-87 1037.8419949637437
-86 1037.8419949637437
-85 1037.8419949637437
-84 1037.8419949637437
-83 1037.8419949637437
-82 1037.8419949637437
-81 1037.8419949637437
-80 1037.8419949637437
-79 1037.8419949637437
-78 1037.8419949637437
-77 1037.5371979214717
-76 1037.5371979214717
-75 1037.5371979214717
-74 1037.5371979214717
-73 1037.5371979214717
-72 1037.5371979214717
-71 1037.5371979214717
-70 1037.5371979214717
-69 1037.5371979214717
-68 1037.5371979214717
-67 1037.4061034308627
-66 1037.2024687400167
-65 1037.1990913267223
-64 1037.1937776093087
-63 1037.189141774531
-62 1037.1787380059432
-61 1037.1379680085681
-60 1037.0936033864937
-59 1037.052701657557
-58 1037.0181952401579
-57 1037.0048754775419
-56 1036.9872422904698
-55 1036.9522360586927
-54 1036.921023349416
-53 1036.812189598139
-52 1036.6865353739404
-51 1036.5270094095297
-50 1036.4855784600948
-49 1036.4199434087618
-48 1036.359674774095
-47 1036.155718845946
-46 1036.0708224938874
-45 1036.00657970

In [221]:
SIGMA2 = xr.open_dataarray(f"./SIGMA2_full.nc")
SIGMA2_ktmin = (SIGMA2.min(dim=["time"])*maskC).copy()
SIGMA2_ktmin = SIGMA2_ktmin.where(SIGMA2_ktmin>1010,other=np.nan)
#SIGMA2_ktmin = SIGMA2_ktmin.where(SIGMA2_ktmin>1010,other=np.nan)
SIGMA2_ktmin = SIGMA2_ktmin.min(dim="k")



In [184]:
SO_sig_mask.to_netcdf("SO_sig2_mask.nc")

In [190]:
ATL_sig_maxes = [[],[]]

In [191]:

# make a blank mask of 1's
blank_mask = xr.open_dataarray("Good_Outputs/T_TH_advection_1.nc").assign_coords(maskC.coords)*0+1
ATL_sig_mask = blank_mask.copy()
IP_sig_mask = blank_mask.copy()

ATL_sig_maxes = [[],[]]
IP_sig_maxes = [[],[]]


for lat in np.arange(30,88):
    
    SIGMA2_k0_tmax_NH_tmp = SIGMA2_k0_tmax_NH.where(SIGMA2_k0_tmax_NH.lat <= lat,other=np.nan).copy()
    sig_max = SIGMA2_k0_tmax_NH_tmp.max()
    print(lat, sig_max.values)
    ATL_sig_maxes[0].append(lat)
    ATL_sig_maxes[1].append(sig_max.values)
    
    tmp_mask = blank_mask.copy()*0
    tmp_mask = tmp_mask.where(tmp_mask.sig < sig_max, other=np.nan)
    tmp_mask = tmp_mask.where(tmp_mask.lat > lat, other=np.nan)
    tmp_mask = tmp_mask.fillna(1)
    
    ATL_sig_mask = ATL_sig_mask*tmp_mask


    
for lat in np.arange(30,88):
    
    SIGMA2_k0_tmax_NH_tmp = SIGMA2_k0_tmax_NH_ip.where(SIGMA2_k0_tmax_NH_ip.lat <= lat,other=np.nan).copy()
    sig_max = SIGMA2_k0_tmax_NH_tmp.max()
    print(lat, sig_max.values)
    IP_sig_maxes[0].append(lat)
    IP_sig_maxes[1].append(sig_max.values)
    
    tmp_mask = blank_mask.copy()*0
    tmp_mask = tmp_mask.where(tmp_mask.sig < sig_max, other=np.nan)
    tmp_mask = tmp_mask.where(tmp_mask.lat > lat, other=np.nan)
    tmp_mask = tmp_mask.fillna(1)
    
    IP_sig_mask = IP_sig_mask*tmp_mask

30 nan
31 1035.4405703284579
32 1035.463412064526
33 1035.510444353676
34 1035.5236133235483
35 1035.5236133235483
36 1035.596299629516
37 1035.659659195692
38 1035.659659195692
39 1035.680386778455
40 1035.7592366216213
41 1035.7656214359376
42 1035.831016795282
43 1035.8851567729807
44 1035.8851567729807
45 1036.0469114215957
46 1036.343696027624
47 1036.4208801046545
48 1036.6086883425128
49 1036.7361459108993
50 1036.9036197892772
51 1036.9247712136023
52 1036.9598060349813
53 1036.9598060349813
54 1036.9598060349813
55 1036.9598060349813
56 1036.9598060349813
57 1036.9598060349813
58 1036.9645045876741
59 1036.9718640113178
60 1036.9774367467928
61 1036.9845945671725
62 1037.0247789667
63 1037.0247789667
64 1037.0247789667
65 1037.0247789667
66 1037.0247789667
67 1037.0247789667
68 1037.0247789667
69 1037.0247789667
70 1037.0247789667
71 1037.0247789667
72 1037.0247789667
73 1037.0247789667
74 1037.0247789667
75 1037.0247789667
76 1037.0247789667
77 1037.0247789667
78 1037.0247789

In [263]:
mod_FINAL_atl_maskC.to_netcdf("mod_FINAL_atl_maskC.nc")

In [243]:
blank_mask = xr.open_dataarray("Good_Outputs/T_TH_advection_1.nc").assign_coords(maskC.coords)*0+1
ATL_sig_kt_mask = blank_mask.copy()
ATL_sig_kt_maxes = [[],[]]
ATL_sig_kt_mins = [[],[]]

for lat in np.arange(0,88):
    
    SIGMA2_ktmax_NH_tmp = SIGMA2_ktmax_NH.where(SIGMA2_ktmax_NH.lat > lat-1,other=np.nan).where(SIGMA2_ktmax_NH.lat <= lat,other=np.nan).copy()
    SIGMA2_ktmin_NH_tmp = (SIGMA2_ktmin*mod_FINAL_atl_maskC).where(SIGMA2_ktmax_NH.lat > lat-1,other=np.nan).where(SIGMA2_ktmax_NH.lat <= lat,other=np.nan).copy()
    sig_max = SIGMA2_ktmax_NH_tmp.max()
    sig_min = SIGMA2_ktmin_NH_tmp.min()
    print(lat, sig_max.values,sig_min.values)
    ATL_sig_kt_maxes[0].append(lat)
    ATL_sig_kt_maxes[1].append(sig_max.values)
    ATL_sig_kt_mins[0].append(lat)
    ATL_sig_kt_mins[1].append(sig_min.values)

0 nan 1027.4061788129827
1 1037.1519247184756 1027.2844241255923
2 1037.1509111413934 1026.0836120561319
3 1037.1491297798232 1025.9977776309233
4 1037.14773243423 1026.089300018202
5 1037.1175647573868 1026.33439336835
6 1037.1142810411884 1027.4295652415467
7 1037.1115746637477 1028.2000770476586
8 1037.1104497126148 1028.645337310277
9 1037.114074754744 1028.8145993768112
10 1037.1127919482742 1028.949414504329
11 1037.1125182319697 1028.8864635714467
12 1037.112541796612 1028.8390120988815
13 1037.1130513468252 1028.831609454271
14 1037.1134131327901 1028.9320751396122
15 1037.1115454641445 1029.060593920098
16 1037.1117885188198 1029.228509617598
17 1037.1136729922994 1029.4297211897638
18 1037.114794992234 1029.5932225632648
19 1037.1150846755072 1029.7657556071088
20 1037.1159815990516 1029.9446146618232
21 1037.1153146061808 1030.0458401643582
22 1037.115027743048 1030.1932177871993
23 1037.114844006787 1030.2205533157844
24 1037.1147006457306 1030.286462748774
25 1037.11480093

In [246]:
ATL_sig_maxes_indepth_da = xr.DataArray(data=np.asarray(ATL_sig_kt_maxes[1]),dims=["lat"],coords={"lat":ATL_sig_kt_maxes[0]})
ATL_sig_mins_indepth_da = xr.DataArray(data=np.asarray(ATL_sig_kt_mins[1]),dims=["lat"],coords={"lat":ATL_sig_kt_mins[0]})


#ATL_sig_maxes_indepth_da.to_netcdf("ATL_sig_maxes_indepth1.nc")
ATL_sig_mins_indepth_da.to_netcdf("ATL_sig_mins_indepth1.nc")

In [ ]:
ATL_sig_maxes_indepth_da.lat

In [ ]:
ATL_sig_maxes_da = xr.DataArray(data=np.asarray(ATL_sig_maxes[1]),dims=["lat"],coords={"lat":ATL_sig_maxes[0]})
IP_sig_maxes_da = xr.DataArray(data=np.asarray(IP_sig_maxes[1]),dims=["lat"],coords={"lat":IP_sig_maxes[0]})

In [119]:
ATL_sig_maxes_da.to_netcdf("ATL_sig_maxes1.nc")
IP_sig_maxes_da.to_netcdf("IP_sig_maxes.nc")

In [244]:
ATL_sig_maxes_da.to_netcdf("ATL_sig_maxes1.nc")

# Make a high res mask

In [36]:
sig2_lvls = np.arange(1036.9,1037.3,0.005)

# Values for "tiles", "i_vals", and "j_vals" are 
tiles=np.arange(0,13)
i_vals=np.arange(0,90)
j_vals=np.arange(0,90)
# set dimensions based on input dataset with sigma-space vertical levels..
ntv_pdens_dims = (len(tiles),
                 len(sig2_lvls),
                 len(j_vals),
                 len(i_vals),
                 )
new_coords = [tiles,sig2_lvls,i_vals,j_vals]
new_dims = ["tile","sig","j","i",]

blank_mask = xr.DataArray(data=np.zeros(ntv_pdens_dims),coords=new_coords,dims=new_dims).assign_coords(maskC.coords)*0+1

SO_sig2_mask = blank_mask.copy()
ATL_sig2_mask = blank_mask.copy()


for lat in np.arange(-88,-20):
    
    SIGMA2_k0_tmax_SH_tmp = SIGMA2_k0_tmax_SH.where(SIGMA2_k0_tmax_SH.lat >= lat,other=np.nan).copy()
    sig_max = SIGMA2_k0_tmax_SH_tmp.max()
    print(lat, sig_max.values)

    tmp_mask = blank_mask.copy()*0
    tmp_mask = tmp_mask.where(tmp_mask.sig < sig_max, other=np.nan)
    tmp_mask = tmp_mask.where(tmp_mask.lat < lat, other=np.nan)
    tmp_mask = tmp_mask.fillna(1)
    
    SO_sig2_mask = SO_sig2_mask*tmp_mask

for lat in np.arange(30,88):
    
    SIGMA2_k0_tmax_NH_tmp = SIGMA2_k0_tmax_NH.where(SIGMA2_k0_tmax_NH.lat <= lat,other=np.nan).copy()
    sig_max = SIGMA2_k0_tmax_NH_tmp.max()
    print(lat, sig_max.values)

    tmp_mask = blank_mask.copy()*0
    tmp_mask = tmp_mask.where(tmp_mask.sig < sig_max, other=np.nan)
    tmp_mask = tmp_mask.where(tmp_mask.lat > lat, other=np.nan)
    tmp_mask = tmp_mask.fillna(1)
    
    ATL_sig2_mask = ATL_sig2_mask*tmp_mask
    
    
    
SO_sig2_mask.to_netcdf("SO_sig2_mask_highres.nc")

(ATL_sig2_mask*FINAL_atl_maskC).to_netcdf("ATL_sig2_mask_highres.nc")

-88 1037.8419949637437
-87 1037.8419949637437
-86 1037.8419949637437
-85 1037.8419949637437
-84 1037.8419949637437
-83 1037.8419949637437
-82 1037.8419949637437
-81 1037.8419949637437
-80 1037.8419949637437
-79 1037.8419949637437
-78 1037.8419949637437
-77 1037.5371979214717
-76 1037.5371979214717
-75 1037.5371979214717
-74 1037.5371979214717
-73 1037.5371979214717
-72 1037.5371979214717
-71 1037.5371979214717
-70 1037.5371979214717
-69 1037.5371979214717
-68 1037.5371979214717
-67 1037.4061034308627
-66 1037.2024687400167
-65 1037.1990913267223
-64 1037.1937776093087
-63 1037.189141774531
-62 1037.1787380059432
-61 1037.1379680085681
-60 1037.0936033864937
-59 1037.052701657557
-58 1037.0181952401579
-57 1037.0048754775419
-56 1036.9872422904698
-55 1036.9522360586927
-54 1036.921023349416
-53 1036.812189598139
-52 1036.6865353739404
-51 1036.5270094095297
-50 1036.4855784600948
-49 1036.4199434087618
-48 1036.359674774095
-47 1036.155718845946
-46 1036.0708224938874
-45 1036.00657970

In [38]:
SO_sig_mask.to_netcdf("SO_sig_mask.nc")

#(ATL_sig_mask*FINAL_atl_maskC).to_netcdf("ATL_sig_mask.nc")

In [39]:
lat = -60
sig_max = 1036.9
tmp_mask = blank_mask.copy().assign_coords(maskC.coords)*0

tmp_mask = tmp_mask.where(tmp_mask.sig < sig_max, other=np.nan)
tmp_mask = tmp_mask.where(tmp_mask.lat < lat, other=np.nan)
tmp_mask = tmp_mask.fillna(1)
#SO_sig_mask.where((SO_sig_mask.sig<sig_max) and maskC.lat<lat)